In [1]:
import numpy as np
import pandas as pd

In [4]:
# year:数据年份
year = '2010-water'

In [5]:
file_name = year+'-node-edge-V20170321'
node_sheetname = year+'-node'
edge_sheetname = year+'-edge'


In [7]:
# 载入数据
node_data = pd.read_excel('D:\qq\\'+file_name+'.xlsx',
                          sheetname=node_sheetname)
edge_data = pd.read_excel('D:\qq\\'+file_name+'.xlsx',
                          sheetname=edge_sheetname)


In [8]:
# 节点数和边数
acounts_node = node_data['Id'].notnull().sum()
acounts_edge = edge_data['Id'].notnull().sum()

In [9]:
# 构建节点和边
node = []    #节点
edge = []    #边
for i in range(acounts_node):
    node_loc = [0,0]    #节点坐标    
    node_loc[0] = int(node_data['X'][i]*10000)
    node_loc[1] = int(node_data['Y'][i]*10000)
    node.append(tuple(node_loc))
for i in range(acounts_edge):
    node_loc = [0,0]    #节点坐标
    edge_loc = [0,0,0]    #边坐标
    node_loc[0] = int(edge_data['X1'][i]*10000)
    node_loc[1] = int(edge_data['Y1'][i]*10000)    
    edge_loc[0] = tuple(node_loc)
    node_loc = [0,0]    #节点坐标    
    node_loc[0] = int(edge_data['X2'][i]*10000)
    node_loc[1] = int(edge_data['Y2'][i]*10000)
    edge_loc[1] = tuple(node_loc)
    edge_loc[2] = edge_data['Length'][i]
    edge.append(tuple(edge_loc))
 

In [10]:
#载入networkx
import networkx as nx

MG = nx.MultiGraph()
MG.add_nodes_from(node)
MG.add_weighted_edges_from(edge)

In [11]:
#开始计算参数
ki_networkx = []
for i in range(acounts_node):
    ki_networkx.append(0)
    ki_networkx[i] = len(MG.neighbors(node[i]))

n_networkx = []
for i in range(acounts_node):
    n_networkx.append(0)
    for n_dicts in MG[node[i]]:
        n_networkx[i] += len(n_dicts)/2

C_networkx = []    #聚集系数C=(2*n)/(ki*(ki-1))
for i in range(acounts_node):
    C_networkx.append(0)
    if ki_networkx[i]>1:
        C_networkx[i] = float(2*n_networkx[i])/(ki_networkx[i]*(ki_networkx[i]-1))
#计算特征路径长度Lp=(2*sum(d_ij))/(n*(n+1))
Lp_no_length_networkx = []  #特征路径长度Lp=(2*sum(d_ij))/(n*(n+1)). d_ij:i到j的最短路径长度
Lp_length_networkx = []  #特征路径长度Lp=(2*sum(d_ij))/(n*(n+1)). d_ij:i到j的最短路径长度
d_ij_no_length_networkx = []
d_ij_length_networkx = []
#无长度图
d_ij_no_length_networkx = nx.shortest_path_length(MG)
for i in range(acounts_node):
    Lp_no_length_networkx.append(0)
    d_ij_no_length_sum_networkx = 0
    for j in range(acounts_node):
        if i != j and node[j] in d_ij_no_length_networkx[node[i]]:
            d_ij_no_length_sum_networkx += d_ij_no_length_networkx[node[i]][node[j]]
    if n_networkx[i]>0:
        Lp_no_length_networkx[i] = float(2*d_ij_no_length_sum_networkx)/(n_networkx[i]*(n_networkx[i]+1))
        
#有长度图
d_ij_length_networkx = nx.shortest_path_length(MG,weight='weight')
for i in range(acounts_node):
    Lp_length_networkx.append(0)
    d_ij_length_sum_networkx = 0
    for j in range(acounts_node):
        if i != j and node[j] in d_ij_length_networkx[node[i]] :
            d_ij_length_sum_networkx += d_ij_length_networkx[node[i]][node[j]]
    if n_networkx[i]>0:
        Lp_length_networkx[i] = float(2*d_ij_length_sum_networkx)/(n_networkx[i]*(n_networkx[i]+1))

# 计算中介中心度Gm=sum(sigma_ab_m/sigma_ab))
#无长度图
Gm_no_length = []
for m in range(acounts_node): 
    Gm_no_length.append(0)
    Gm_ab_no_length = 0
    sigma_ab_m_no_length = 0
    sigma_ab_no_length = 0
    for i in range(acounts_node): 
        for j in range(acounts_node):
            if i > j and i!=m and j!=m and node[j] in d_ij_no_length_networkx[node[i]]:
                paths_ab_no_length = [p for p in nx.all_shortest_paths(MG,source=node[i],target=node[j])]
                sigma_ab_no_length += len(paths_ab_no_length)
                for path in paths_ab_no_length:
                    if node[m] in path:
                        sigma_ab_m_no_length += 1
                Gm_ab_no_length += sigma_ab_m_no_length/sigma_ab_no_length
                sigma_ab_m_no_length = 0
                sigma_ab_no_length = 0                   
    Gm_no_length[m] = Gm_ab_no_length
    Gm_ab_no_length = 0
#有长度图   
Gm_length = []
for m in range(acounts_node): 
    Gm_length.append(0)
    Gm_ab_length = 0
    sigma_ab_m_length = 0
    sigma_ab_length = 0
    for i in range(acounts_node): 
        for j in range(acounts_node):
            if i > j and i!=m and j!=m and node[j] in d_ij_length_networkx[node[i]]:
                paths_ab_length = [p for p in nx.all_shortest_paths(MG,source=node[i],target=node[j],weight='weight')]
                sigma_ab_length += len(paths_ab_length)
                for path in paths_ab_length:
                    if node[m] in path:
                        sigma_ab_m_length += 1
                Gm_ab_length += sigma_ab_m_length/sigma_ab_length
                sigma_ab_m_length = 0
                sigma_ab_length = 0                   
    Gm_length[m] = Gm_ab_length
    Gm_ab_length = 0    

# 将参数写入文件
model = pd.read_csv('D:\qq\model.csv')
model['node'] = node_data['Id'][:acounts_node]
model['X'] = node_data['X'][:acounts_node]
model['Y'] = node_data['Y'][:acounts_node]
model['ki'] = np.array(ki_networkx)
model['n'] = np.array(n_networkx)
model['C'] = np.array(C_networkx)
model['Lp_no_length'] = np.array(Lp_no_length_networkx)
model['Lp_length'] = np.array(Lp_length_networkx)
model['Gm_no_length'] = np.array(Gm_no_length)
model['Gm_length'] = np.array(Gm_length)
model.to_csv('D:\qq\\results_'+year+'-node-edge.csv')

KeyboardInterrupt: 